# Image Enhancement & Background Removal
This notebook provides tools to fix blurry/pixelated images and remove backgrounds cleanly.

**Features:**
- **Super Resolution (Upscaling):** Fixes old or small photos to make them high-definition.
- **Background Removal:** Extracts the main subject from an image automatically.
- **Line-by-Line Documentation:** Beginner-friendly explanations for every step.

In [ ]:
# Step 1: Install specialized AI tools for image processing
# %pip is a command to download and install Python tools efficiently in a notebook
# 'transformers' handles the AI models from Hugging Face
# 'onnxruntime' help some models run even faster
%pip install -q transformers torch Pillow ipywidgets numpy

In [ ]:
# Import the core engine 'torch' for AI math
import torch
# 'Image' is our main tool for opening and seeing pictures
from PIL import Image
# 'AutoModelForImageSegmentation' is a specialized tool for background removal
from transformers import AutoModelForImageSegmentation, AutoImageProcessor
# 'pipeline' is the easiest way to load a specific AI task
from transformers import pipeline
# UI tools to create buttons and uploaders
import ipywidgets as widgets
# Tools to show images in the notebook
from IPython.display import display
import io
import numpy as np

# Detect if there is a fast Graphics Card (GPU) available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Hardware detected: {device}")

In [ ]:
# --- Defining the Models ---

# The 'RMBG' model is famous for its accuracy in removing backgrounds
BG_MODEL_ID = "briaai/RMBG-1.4"

# The 'Swin2SR' model is great at reconstruction (fixing blurry parts)
UPSCALER_MODEL_ID = "caidas/swin2SR-classical-sr-x2-64"

# We will load these models 'on demand' later to save memory

In [ ]:
# --- Logic for Background Removal ---
def remove_background(img):
    print("Loading Background Removal Model...")
    # Load the specific processor and model for this task
    # IMPORTANT: We use trust_remote_code=True because this model uses a specialized internal processor
    processor = AutoImageProcessor.from_pretrained(BG_MODEL_ID, trust_remote_code=True)
    model = AutoModelForImageSegmentation.from_pretrained(BG_MODEL_ID, trust_remote_code=True)
    model.to(device) # Move to GPU/CPU
    
    # Prepare the image for the AI (resizing and normalizing it)
    inputs = processor(img, return_tensors="pt").to(device)
    
    # Predict which pixels are background vs foreground
    with torch.no_grad():
        logits = model(**inputs).logits
    
    # Resize the prediction (mask) to match the original image size exactly
    mask = torch.nn.functional.interpolate(logits, size=img.size[::-1], mode='bilinear', align_corners=False)
    # Convert high-level math into a simple 0 to 1 range (Alpha channel)
    mask = torch.sigmoid(mask).cpu().numpy().squeeze()
    
    # Apply the mask to create a transparent image (RGBA)
    mask_image = Image.fromarray((mask * 255).astype('uint8'), mode='L')
    img_rgba = img.convert("RGBA")
    img_rgba.putalpha(mask_image)
    
    return img_rgba

# --- Logic for Upscaling ---
def upscale_image(img):
    print("Loading Upscaler Model (making it HD)...")
    # 'image-to-image' pipeline is used for upscaling / super-resolution
    upscaler = pipeline("image-to-image", model=UPSCALER_MODEL_ID, device=0 if device == "cuda" else -1)
    
    # Process the image to double its size and fix details
    result = upscaler(img)
    
    # FIX: Some versions of transformers return a list, others return the image directly
    # If it's a list, we take the first item. If not, we just return it as is.
    if isinstance(result, list):
        return result[0]
    return result

In [ ]:
# --- Interactive Enhancement UI ---

# Create an uploader for your photo
uploader = widgets.FileUpload(accept='image/*', multiple=False, description="Upload Photo")

# Create a choice for what task to perform
task_selector = widgets.Dropdown(
    options=[('Upscale (Fix Pixels)', 'upscale'), ('Remove Background', 'remove_bg')],
    value='upscale',
    description='Action:',
)

button = widgets.Button(description="Process Image", button_style='success', layout=widgets.Layout(width='200px'))
output = widgets.Output()

def on_click(b):
    with output:
        output.clear_output()
        if not uploader.value:
            print("Please upload a photo first!")
            return
        
        # Extract the image data from the uploader
        uploaded_file = list(uploader.value.values())[0] if isinstance(uploader.value, dict) else uploader.value[0]
        input_image = Image.open(io.BytesIO(uploaded_file['content']))
        
        print(f"Processing: {task_selector.label}...")
        
        try:
            # Choose which function to run based on the user's choice
            if task_selector.value == 'upscale':
                final_image = upscale_image(input_image)
            else:
                final_image = remove_background(input_image)
            
            # Show the final result
            print("Done! Displaying result:")
            display(final_image)
        except Exception as e:
            print(f"An error occurred during processing: {e}")
            if "RMBG" in str(e):
                print("TIP: This might be an environment issue. Try restarting the session and running Step 1 again.")
            elif "subscriptable" in str(e):
                print("FIX APPLIED: If you see this error, please re-run the 'Logic' cell and try again.")

button.on_click(on_click)

# Display the UI nicely arranged
ui_box = widgets.VBox([uploader, task_selector, button, output])
display(ui_box)